#### MESHpy Test

##### Turn Autosave off
Autosave generates additional work for the version control. Remember to manually save any chances after clearing kernel and output for more effective version control.

In [ ]:
%autosave 0 

##### Google Colab execution
Using this Chrome extension, the github-hosted jupyter notebooks may be opened directly in Google Colaboratory
https://chrome.google.com/webstore/detail/open-in-colab/iogfkhleblhcpcekbiedikdehleodpjo
more info here:
https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb

The following two StackOverflow posts helped with managing the dependencies
https://stackoverflow.com/questions/53581278/test-if-notebook-is-running-on-google-colab
https://stackoverflow.com/questions/53793731/using-custom-packages-on-google-colaboratory

[![Open This Notebook In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jameshalgren/wrf_hydro_nwm_public/blob/dynamic_channel_routing/trunk/NDHMS/dynamic_channel_routing/MESHpyNetwork.ipynb)

In [ ]:
try:
    import google.colab
    IN_COLAB = True
    !git clone https://github.com/jameshalgren/wrf_hydro_nwm_public.git
    import sys
    sys.path.append('/content/wrf_hydro_nwm_public/trunk/NDHMS/dynamic_channel_routing')
except:
    IN_COLAB = False

###### Note
9 Sept 2016
The plotting functions seem to require that the cell be executed twice before the output will appear in a standard Jupyter Notebook. 

In [ ]:
from __future__ import division
import pickle
from MESHpyNetwork import MESHpyNetwork
import matplotlib.pyplot as plt
from math import ceil
import pandas as pd
import os

def main():
    input_type = 'file'
    input_vars = {}
    input_vars['filetype'] = 'mesh.py'
    root = os.path.abspath(r'c:/Users/james.halgren/Downloads/MESH_test/')
    Main_Example_Path = os.path.join(root , 'US')
    Sub_Example_Path = os.path.join(Main_Example_Path , 'BW')
    This_Example_Path = os.path.join(Sub_Example_Path, 'Q')

    #C:\Users\james.halgren\Downloads\MESH_test\US\BW\Q\Qvar_us_2YNorm\Qvar_us_0033_5.0-10000.0_0100_0000-0004-0200_2NormalDepth
    input_path = os.path.join(This_Example_Path,'Qvar_us_2YNorm','Qvar_us_0033_5.0-10000.0_0100_0000-0004-0200_2NormalDepth',"input.txt")
    input_vars['input_path'] = input_path
    # print(input_path)
    
    # if len(sys.argv) > 1:
    #     input_path = sys.argv[1]
    # else:
    #     input_path = os.path.join(This_Example_Path,"input.txt")
    #     print(input_path)
    #     #input_path = "./input.txt"

    # network = DummyNetwork()
    # network = SimpleFlowTrace() #DongHa's method.
    # network = SteadyNetwork(input_type = input_type, input_vars = input_vars)
    #input_and_initialize(sections, input_path, input_opt)
    network = MESHpyNetwork(input_type = input_type, input_vars = input_vars)
    # network = MuskCNetwork()
    # network = MESHDNetwork()

    network.compute_initial_state()
    network.compute_time_steps()

    cols_for_subplots = 4
    fig, axes = plt.subplots(nrows=ceil(len(network.sections)/cols_for_subplots), ncols=cols_for_subplots, squeeze=False)

    #TODO: make plotting another method within the network
    for i, section in enumerate(network.sections):
        '''Section Hydrograph Plots'''
        a = pd.Series(time_step.depth for i, time_step in enumerate(section.time_steps))
        # print(m, n)
        m = i//cols_for_subplots
        n = i%cols_for_subplots 
        a.plot(ax = axes[m,n])
        
    cols_for_subplots = 4
    fig2, axes2 = plt.subplots(nrows=ceil(len(network.sections[0].time_steps)/cols_for_subplots), ncols=cols_for_subplots, squeeze=False)

    #TODO: make plotting another method within the network
    for j, time_step in enumerate(network.sections[0].time_steps):
        '''Profile Plots'''
        b = pd.Series(section.bottom_z + section.time_steps[j].depth for i, section in enumerate(network.sections))
        # print(m, n)
        m = j//cols_for_subplots
        n = j%cols_for_subplots 
        # Thanks to this SO post for the easy indexing solution:
        # https://stackoverflow.com/questions/5494974/convert-1d-array-index-to-2d-array-index
        b.plot(ax = axes2[m,n])

    cols_for_subplots = 4
    fig3, axes3 = plt.subplots(nrows=ceil(len(network.sections[0].time_steps)/cols_for_subplots), ncols=cols_for_subplots, squeeze=False)
        
    #TODO: make plotting another method within the network
    for j, time_step in enumerate(network.sections[0].time_steps):
        '''Profile Plots'''
        c = pd.Series(section.time_steps[j].depth for i, section in enumerate(network.sections))
        # print(m, n)
        m = j//cols_for_subplots
        n = j%cols_for_subplots 
        c.plot(ax = axes3[m,n])
        
    #TODO: Use this generator to create a get_depths method of some kind within the Network class.
    #      Such a method could replace the a = pd.Series... and b = pd.Series... lines in the code above.
    elevations = [[section.bottom_z + section.time_steps[j].depth for section in network.sections] for j, _ in enumerate(network.sections[0].time_steps)]
    depths = [[section.time_steps[j].depth for section in network.sections] for j, _ in enumerate(network.sections[0].time_steps)]
    #print(depths)

    return network

if __name__ == "__main__":
    network = main()
    pickle.dumps(network.sections)